###Full name: **Trương Sĩ Thi Vũ**
###Student number: **USTHBI8-189**
###Email address: **vutst.b8189@st.usth.edu.vn**

Please run each cell to observe the final results. (Entropy: 11.284992503675998
Perplexity: 2495.2912543003954)

## Data

We will use the file [wiki-en-train.word](https://raw.githubusercontent.com/neubig/nlptutorial/master/data/wiki-en-train.word) as the training data, and [wiki-en-test.
word](https://raw.githubusercontent.com/neubig/nlptutorial/master/data/wiki-en-test.word) as the test data. To test our implementation quickly, we will use small data file [02-train-input.txt](https://github.com/neubig/nlptutorial/blob/master/test/02-train-input.txt). All data files are from the [nlptutorial](https://github.com/neubig/nlptutorial) by Graham Neubig.

As the first step, we will download all necessary data files using `wget` command line.

In [0]:
!rm -f wiki-en-train.word
!wget https://raw.githubusercontent.com/neubig/nlptutorial/master/data/wiki-en-train.word
    
!rm -f wiki-en-test.word
!wget https://raw.githubusercontent.com/neubig/nlptutorial/master/data/wiki-en-test.word

!rm -f 02-train-input.txt
!wget https://raw.githubusercontent.com/neubig/nlptutorial/master/test/02-train-input.txt

--2020-02-08 16:55:56--  https://raw.githubusercontent.com/neubig/nlptutorial/master/data/wiki-en-train.word
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 203886 (199K) [text/plain]
Saving to: ‘wiki-en-train.word’

wiki-en-train.word  100%[===================>] 199.11K  --.-KB/s    in 0.04s   

2020-02-08 16:55:58 (4.35 MB/s) - ‘wiki-en-train.word’ saved [203886/203886]

--2020-02-08 16:56:07--  https://raw.githubusercontent.com/neubig/nlptutorial/master/data/wiki-en-test.word
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26989 (26K) [t

## Part 1: Estimating probabilities

What you need to do in this part is complete the function `train_bigram` to estimate unigram, bigram probabilities (using MLE method) from a text file and save probabilities to a model file.

The format of the model file is as follows. Each line includes an n-gram (unigram or bigram) and its probability estimated by MLE method.

```
<s> a	1.000000
a	0.250000
a b	1.000000
b	0.250000
b c	0.500000
...
```

### Part 1.1: Function train_bigram()

In [0]:
from collections import defaultdict


def train_bigram(train_file, model_file):
    """Train trigram language model and save to model file
    """
    counts = defaultdict(int)  # count the n-gram
    word_count = 0 # total words (V)
    context_counts = defaultdict(int)   # count the context
    with open(train_file) as f:
        for line in f:
            line = line.strip()
            if line == '':
                continue
            words = line.split()
            words.append('</s>')
            words.insert(0, '<s>')

    
            for i in range(1, len(words)):  
                word_count += 1
                counts[words[i]] +=1
                counts[words[i-1]+ " "+ words[i]] += 1
                context_counts[words[i-1]] +=1
                # print("The probability of getting {} if already had {} is {}".format(words[i],words[i-1],(counts[words[i-1]+ " " + words[i]]/context_counts[words[i-1]])))
                pass
   


    # Save probabilities to the model file            
    with open(model_file, 'w') as fo:
        for ngram, count in counts.items():
           
            if len(ngram.split()) > 1:
              fo.write(ngram + "\t" + "{}".format(counts[ngram]/context_counts[ngram.split()[0]])+"\n")
            else: fo.write((ngram + "\t" + "{}".format(counts[ngram]/word_count))+"\n")
            
            pass

train_bigram("wiki-en-train.word","02-train-answer.txt")

Let's try to train bigram model on the small data.

In [0]:
train_bigram('02-train-input.txt', '02-train-answer.txt')

Let's see the content of the model. After completing the function `train_bigram`, you should see. The order of lines may be different.

```
</s>	0.250000
<s> a	1.000000
a	0.250000
a b	1.000000
b	0.250000
b c	0.500000
b d	0.500000
c	0.125000
c </s>	1.000000
d	0.125000
d </s>	1.000000
```

In [0]:
!cat 02-train-answer.txt

a	0.25
<s> a	1.0
b	0.25
a b	1.0
c	0.125
b c	0.5
</s>	0.25
c </s>	1.0
d	0.125
b d	0.5
d </s>	1.0


### Part 1.2: load the model file

We are going to implement the function `load_bigram_model` to load the model file.

In [0]:
def load_bigram_model(model_file):
  
    probs = {}
    with open(model_file, 'r') as f:
        for line in f:
            line = line.strip()
            probs[line.split("\t")[0]] = float(line.split("\t")[1])
            pass
    return probs
  


Let's test the function

In [0]:
print(load_bigram_model('02-train-answer.txt'))


{'a': 0.25, '<s> a': 1.0, 'b': 0.25, 'a b': 1.0, 'c': 0.125, 'b c': 0.5, '</s>': 0.25, 'c </s>': 1.0, 'd': 0.125, 'b d': 0.5, 'd </s>': 1.0}


## Part 2: Evaluating Bigram Language Model

In this part, we will evaluate the bigram language model on the test set. We will use linear interpolation as the smoothing technique.

What we need to do is to complete the function  `test_bigram` as follows. The function will return perplexity on the test data.

Recall that, the smoothed bigram probabilities using interpolation technique is calculated as follows.

Bigrams:

$$
P(w_i|w_{i-1})=\lambda_2 \times P_{ML}(w_i|w_{i-1})+(1-\lambda_2)\times P(w_i)
$$

where $P(w_i)$ is the smoothed unigram probability and calculated as follows.

$$
P(w_i)=\lambda_1 \times P_{ML}(w_i) + (1-\lambda_1) \times \frac{1}{N}
$$

where $N$ is a large number, e.g., $N=1,000,000$.

In [0]:
import math
from collections import defaultdict


def test_bigram(test_file, model_file, lambda2=0.95, lambda1=0.95, N=1000000):
    W = 0 # Total word count
    H = 0

    probs = load_bigram_model(model_file)
    with open(test_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line == '':
                continue
            words = line.split()
            words.append('</s>')
            words.insert(0, '<s>')
            for i in range(1, len(words)):  
                if (probs.get(words[i])):
                  p1 = lambda1 * probs.get(words[i]) + (1-lambda1) * (1/N)
                else: 
                  p1 = (1-lambda1) * (1/N) 
                if (probs.get(words[i-1] + " " + words[i])):
                  p2 = lambda2 * probs.get(words[i-1] + " " + words[i])+ (1-lambda2) * p1
                else:
                  p2 = (1-lambda2) * p1

                W += 1  # Count the words
                H += -math.log2(p2) # We use logarithm to avoid underflow
    H = H/W
    P = 2**H
    
    print("Entropy: {}".format(H))
    print("Perplexity: {}".format(P))

    return 

Now let's calculate on the Wikipedia data.

In [0]:
train_bigram('wiki-en-train.word', 'bigram_model.txt')
test_bigram('wiki-en-test.word', 'bigram_model.txt')

Entropy: 11.284859117885485
Perplexity: 2495.0605603552463


You should get entropy value about 11.28.

[link text](https://)